<a href="https://colab.research.google.com/github/zeelShah-12/AI-Skincare-Advisor/blob/main/AI_Skincare_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📦 Install dependencies

In [27]:
!pip install pandas scikit-learn gradio
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr


📚 Imports & 📥 Load data

```
# This is formatted as code
```



In [28]:
import pandas as pd

df = pd.read_csv("MP-Skin Care Product Recommendation System3.csv")

df.head()


,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=..."
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f..."


🧹 Clean data

In [29]:

df['notable_effects'] = df['notable_effects'].fillna('')
df['skintype'] = df['skintype'].fillna('')
df['description'] = df['description'].fillna('')

# Combine relevant features into one text column
df['combined_features'] = df['notable_effects'] + ' ' + df['skintype'] + ' ' + df['description']

# remove duplicates based on product_name
df.drop_duplicates(subset="product_name", inplace=True)

# Preview
df[['product_name', 'combined_features']].head()


,product_name,combined_features
0,ACWELL Bubble Free PH Balancing Cleanser,"Acne-Free, Pore-Care, Brightening, Anti-Aging ..."
1,ACWELL pH Balancing Soothing Cleansing Foam,"Soothing, Balancing Normal, Dry, Combination M..."
2,Acwell Licorice pH Balancing Cleansing Toner,"Soothing, Balancing Normal, Dry, Oily, Combina..."
3,ACWELL Aquaseal Soothing Tonic,"Acne-Free, Pore-Care, Brightening, Anti-Aging ..."
4,Licorice pH Balancing Essence Mist,"Brightening, Soothing Normal, Dry Essens mist ..."


TF-IDF

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF on combined features
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

# Create cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


Save recommendation log

In [44]:
def log_query(skin_type, effect, price_limit, product_type):
    with open("query_logs.csv", "a") as f:
        f.write(f"{datetime.now()},{skin_type},{effect},{price_limit},{product_type}\n")

📤 Export recommendations

In [45]:
def export_csv(df_slice):
    filename = "recommendations.csv"
    df_slice.to_csv(filename, index=False)
    return filename

🧠 Recommendation function

In [49]:
def recommend(skin_type, effect, price_limit, product_type):
    log_query(skin_type, effect, price_limit, product_type)

    input_text = effect + ' ' + skin_type
    input_vec = vectorizer.transform([input_text])
    scores = cosine_similarity(input_vec, tfidf_matrix).flatten()

    df['score'] = scores
    filtered = df[df['price'] <= price_limit]
    if product_type != "Any":
        filtered = filtered[filtered['product_type'] == product_type]

    top = filtered.sort_values("score", ascending=False).head(5)
     # Format for output
    output = ""
    for _, row in top.iterrows():
        output += f"🔹 **[{row['product_name']}]({row['product_href']})**\n"
        output += f"- Brand: {row['brand']}\n"
        output += f"- Type: {row['product_type']}\n"
        output += f"- Skin Type: {row['skintype']}\n"
        output += f"- Effects: {row['notable_effects']}\n"
        output += f"- Price: ${row['price']:.2f}\n"
        output += f"- Description: {row['description'][:200]}...\n\n"
    if output == "":
        output = "❌ No matching products found. Try a different filter."

    return output, export_csv(top)

📱 UI

In [50]:
product_types = ["Any"] + sorted(df['product_type'].unique().tolist())

app = gr.Interface(
    fn=recommend,
    inputs=[
        gr.Radio(["oily", "dry", "normal", "combination", "sensitive"], label="Skin Type"),
        gr.Textbox(label="Desired Effect (e.g. brightening, hydration, anti-acne)"),
        gr.Slider(0, 200, step=5, label="Max Price"),
        gr.Dropdown(product_types, label="Product Type")
    ],
    outputs=[
        gr.Markdown(label="Top Products"),
        gr.File(label="Download Recommendations (CSV)")
    ],
    title="🌿 Skincare Product Recommender",
    description="Get skincare product suggestions based on your skin type, goals, and budget."
)

In [32]:
recommend_products("dry", "hydrating")


,product_name,brand,product_type,notable_effects,skintype,price
818,PYUNKANG YUL Mist Toner (100ml Medium),PYUNKANG YUL,Toner,"Hydrating, Soothing",Dry,Rp 180.000
682,Mediheal Mediheal Aloe Vera Hydrogel,MEDIHEAL,Moisturizer,"Hydrating, Moisturizing","Normal, Dry",Rp 69.000
445,Haple Rice Bran Oil,HAPLE,Moisturizer,"Hydrating, Moisturizing","Normal, Dry, Oily, Combination, Sensitive",Rp 59.000
1071,The Aubree Centella + Greentea Power Cream,THE AUBREE,Moisturizer,"Hydrating, Moisturizing","Normal, Dry, Oily, Combination, Sensitive",Rp 99.000
21,A'pieu Deep Clean Cleansing Water,A'PIEU,Face Wash,"Hydrating, Moisturizing","Normal, Dry",Rp 89.000


In [33]:
!pip install gradio --quiet


In [34]:
from textblob import TextBlob

# Install if needed
# !pip install textblob

# Apply sentiment polarity
df['sentiment_score'] = df['description'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [35]:
df.sort_values('sentiment_score', ascending=False)[['product_name', 'brand', 'sentiment_score']].head()


,product_name,brand,sentiment_score
36,Avoskin PHTE Special Edition Avoskin x Sociolla,AVOSKIN,1.0
165,BIO BEAUTY LAB Luxurious Face Oil,BIO BEAUTY LAB,1.0
1170,Wardah Acnederm Night Treatment Moisturizer,WARDAH,1.0
950,SKINMEE Treasure Perfect Radiance Serum,SKINMEE,1.0
164,EXCLUSIVEBio Beauty Lab Luxurious Face Oil,BIO BEAUTY LAB,1.0


In [36]:
def gradio_recommend(skin_type, effect, price_limit):
    results = recommend_products(skin_type, effect)
    results = results[results['price'] <= price_limit]
    return results.to_markdown(index=False)

iface = gr.Interface(
    fn=gradio_recommend,
    inputs=[
        gr.Radio(["oily", "dry", "normal", "combination", "sensitive"], label="Your Skin Type"),
        gr.Textbox(label="Desired Effect (e.g. hydrating, brightening)"),
        gr.Slider(minimum=0, maximum=200, step=5, label="Max Price")
    ],
    outputs="markdown",
    title="🌿 Skincare Product Recommender",
    description="Find products that match your skin type, desired result, and budget!"
)


In [37]:
def gradio_recommend(skin_type, effect, price_limit):
    results = recommend_products(skin_type, effect)
    results = results[results['price'] <= price_limit]
    return results.to_markdown(index=False)

iface = gr.Interface(
    fn=gradio_recommend,
    inputs=[
        gr.Radio(["oily", "dry", "normal", "combination", "sensitive"], label="Your Skin Type"),
        gr.Textbox(label="Desired Effect (e.g. hydrating, brightening)"),
        gr.Slider(minimum=0, maximum=200, step=5, label="Max Price")
    ],
    outputs="markdown",
    title="🌿 Skincare Product Recommender",
    description="Find products that match your skin type, desired result, and budget!"
)


In [38]:
product_types = ['Any'] + sorted(df['product_type'].dropna().unique().tolist())

iface = gr.Interface(
    fn=gradio_recommend,
    inputs=[
        gr.Radio(["oily", "dry", "normal", "combination", "sensitive"], label="Your Skin Type"),
        gr.Textbox(label="Desired Effect (e.g. hydrating, brightening)"),
        gr.Slider(minimum=0, maximum=200, step=5, label="Max Price"),
        gr.Dropdown(product_types, label="Product Type")
    ],
    outputs="markdown",
    title="🌿 Skincare Product Recommender",
    description="Find products that match your skin type, desired result, and budget!"
)


In [39]:
def gradio_recommend(skin_type, effect, price_limit, product_type):
    results = recommend_products(skin_type, effect)
    results = results[results['price'] <= price_limit]

    if product_type != 'Any':
        results = results[results['product_type'] == product_type]

    return results.to_markdown(index=False)


In [40]:
def format_with_links(df_slice):
    return "\n".join([
        f"[{row['product_name']} - {row['brand']}]({row['product_href']})"
        for _, row in df_slice.iterrows()
    ])


In [41]:
import datetime

def log_query(skin_type, effect):
    with open("query_logs.csv", "a") as file:
        file.write(f"{datetime.datetime.now()},{skin_type},{effect}\n")


In [51]:
app.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://90a3cdcd749406be1e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
